In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def dydx(x,y):
    
    #set derivatives
    
    #our equation is d^2y/dx^2 = -y
    
    #so we can write
    #dydx = z
    #dzdx = -y
    
    #we'll set y = y[0]   (the 1st element in the array)
    #and z = y[1]         (the 2nd element in the array)
    
    #declare the array
    y_derivs = np.zeros(2)
    
    #set dydx = z
    y_derivs[0] = y[1]
    
    #set dzdx = -y
    y_derivs[1] = -1*y[0]
    
    #we now return the array
    return y_derivs

In [ ]:
def ck_mv_core(dydx,xi,yi,N,h):
    #advance f by a step h
        
    #declare k? arrays
    k1 = np.zeros(N)
    k2 = np.zeros(N)
    k3 = np.zeros(N)
    k4 = np.zeros(N)
    k5 = np.zeros(N)
    k6 = np.zeros(N)
    
    #half step (i + 1/2)
    x_ipoh = xi + 0.5*h
    
    #advance x at 1 step
    x_ipo = xi + h
    
    #declare a temp y array
    y_temp = np.zeros(nv)
    
    #get k1 values
    y_derivs = dydx(xi,yi)
    k1[:] = h*y_derivs[:]
    
    #get k2 values
    y_temp[:] = yi[:] + 0.5*k1[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k2[:] = h*y_derivs[:]
    
    #get k3 values
    y_temp[:] = yi + 0.5*k2[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k3[:] = h*y_derivs[:]
    
    #get k4 values
    y_temp[:] = yi + 0.5*k3[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k4[:] = h*y_derivs[:]
    
    #get k5 values
    y_temp[:] = yi + 0.5*k4[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k5[:] = h*y_derivs[:]
    
    #get k6 values
    y_temp[:] = yi + 0.5*k5[:]
    y_derivs = dydx(x_ipo,y_temp)
    k6[:] = h*y_derivs[:]
    
    
    yipo = yi + 37/378*k1 + 250/621*k3 + 125/594*k4 + 512/1771*k6
    
    return yipo

In [ ]:
def ck_mv_ad(dydx, xi, yi, nv, h, tol):
    
    #Define a safety scale
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    #set a max number of iterations to catch infinite loops for the while
    imax = 10000
    
    #set an iteration variable
    i=0
    
    #Create an error
    Delta = np.full(nv, 2*tol)
    
    #remember the step
    h_step = h
    
    
    
    #step adjustment
    while(Delta.max()/tol > 1.0):
        
        #estimate error by taking step size h vs. 2 steps of h/2
        y_2 = ck_mv_core(dydx, xi, yi, nv, h_step)
        y_1 = ck_mv_core(dydx, xi, yi, nv, 0.5*h_step)
        y_11 = ck_mv_core(dydx, xi+0.5*h_step, y_1, nv, 0.5*h_step)
        
        #Compute the error
        Delta = np.fabs(y_2 - y_11)
        
        #if error is too large, take smaller step
        if (Delta.max()/tol > 1.0):
            
            #our error is too large, decrease the step size
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
            
        if(i>=imax):
            print ("Too many iterations in rk4_mv_ad()")
            raise StopIteration("Ending after i = ", i)
        
        #iterate 
        i+=1
        
    #next time, try a bigger step
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9), h_step * H_NEW_FAC)
    
    #return the answer, a new step, and the step we actually took
    return y_2, h_new, h_step

In [ ]:
def ck_mv(dydx,a,b,y_a,tol):
    
    #dydx is deriv wrt x
    #a is lower bound
    #b is upper bound
    #y_a are uppper boundary conditions
    #tol is tolerance for integrating y
    
    #define starting step
    xi = a
    yi = y_a.copy()
    
    #an initial step size -- make it very small!!
    h = 1.0e-4*(b-a)
    
    #set max iteration count
    imax = 10000
    
    #set an iteration variable
    i=0
    
    #set number of coupled ODEs to the size of y_a
    nv = len(y_a)
    
    x = np.full(1,a)
    y = np.full((1,nv),y_a)   #makes an '1 x nv' size matrix array
    
    #set a flag
    flag = 1
    
    while(flag):
        
        #calculate y_i+1
        yi_new, h_new, h_step = ck_mv_ad(dydx,xi,yi,nv,h,tol)
        
        #update the step
        h = h_new
        
        if(xi+h_step>b):
            
            #take a smaller step
            h = b-xi
            
            yi_new, h_new, h_step = ck_mv_ad(dydx,xi,yi,nv,h,tol)
            
            #break
            flag = 0
    
        #update values
        xi += h_step
        yi[:] = yi_new[:]
        
        #add the step to the arrays
        x = np.append(x,xi)
        y_new = np.zeros((len(x),nv))     #matrix size of length of 'x array' by 'nv'
        y_new[0:len(x)-1,:] = y
        y_new[-1,:] = yi[:]               #assigning the to the row '-1' the array 'yi'
        del y
        y = y_new
        
        
        #prevent overflow
        if(i>=imax):
            print("Max iterations reached.")
            raise StopIteration("iteration number: ",i)
        
        #iterate
        i+=1
        
        #output info in a fixed column of width 3 (%3), d is an int, \t is a tab, 9 floating points w/ 8 decimals
        s = "i= %3d\tx = %9.8f\th = %9.8f\tb = %9.8f" % (i,xi,h_step,b)
        print(s)
        
        #break if new xi is b
        if(xi==b):
            flag = 0
    
    #return answer
    return x ,y

In [ ]:
a = 0.0
b = 2.0 *np.pi

y_0 = np.zeros(2)
y_0[0] = 0.0
y_0[1] = 1.0
nv = 2

tol = 1.0e-6

#perform integration
x, y = ck_mv(dydx,a,b,y_0,tol)

In [ ]:
plt.plot(x,y[:,0],'o',label='y(x)')
plt.plot(x,y[:,1],'o',label='dydx(x)')
xx = np.linspace(0,2.0*np.pi,1000)
plt.plot(xx,np.sin(xx),label='sin(x)')
plt.plot(xx,np.cos(xx),label='cos(x)')
plt.xlabel('x')
plt.ylabel('y')
plt.legend(frameon=0)

In [ ]:
sine = np.sin(x)
cosine = np.cos(x)

y_error = (y[:,0]-sine)
dydx_error = (y[:,1]-cosine)

plt.plot(x,y_error,'o',label='y(x) Error')
plt.plot(x,dydx_error,'o',label='dydx(x) Error')
plt.legend(frameon=0)